In [12]:
import pandas as pd
import os
import numpy as np
from geopy.distance import geodesic
from math import radians, degrees, atan2, sin, cos

In [13]:
os.chdir(os.path.expanduser("~") + "/Developer/Agnirath/TrackTesting")
os.getcwd()

'/Users/kevinkinsey/Developer/Agnirath/TrackTesting'

In [14]:
df = pd.read_csv("./data/raw/raw_route_data.csv", ).drop_duplicates().reset_index(drop=True)
df

,Latitude,Longitude,Elevation (m)
0,13.234640,80.278130,6.0
1,13.234610,80.278040,6.0
2,13.234580,80.277950,6.0
3,13.234550,80.277860,6.0
4,13.234495,80.277775,6.0
...,...,...,...
122,13.234601,80.278549,6.0
123,13.234614,80.278451,6.0
124,13.234626,80.278354,6.0
125,13.234638,80.278257,6.0


In [15]:
lats = df['Latitude'].values
lons = df['Longitude'].values
elevs = df['Elevation (m)'].values
N = len(df)

In [16]:
step_distances = []
directions = []

for i in range(N):
    # Current point
    lat1, lon1 = lats[i], lons[i]
    # Next point (loop back for last)
    lat2, lon2 = lats[(i+1) % N], lons[(i+1) % N]

    # 1️⃣ Compute geodesic distance (meters)
    dist = geodesic((lat1, lon1), (lat2, lon2)).meters
    step_distances.append(dist)
    # slope.append(atan2(elev2 - elev1, dist))

    # 2️⃣ Compute azimuth (bearing) in degrees
    # Formula: https://www.movable-type.co.uk/scripts/latlong.html
    φ1, φ2 = radians(lat1), radians(lat2)
    Δλ = radians(lon2 - lon1)

    X = sin(Δλ) * cos(φ2)
    Y = cos(φ1)*sin(φ2) - sin(φ1)*cos(φ2)*cos(Δλ)
    bearing = atan2(Y, X)
    directions.append(bearing)

In [17]:
df['step_distance'] = step_distances
df['cum_distance'] = np.cumsum(step_distances)
df['direction'] = directions

df

,Latitude,Longitude,Elevation (m),step_distance,cum_distance,direction
0,13.234640,80.278130,6.0,10.303564,10.303564,-2.811680
1,13.234610,80.278040,6.0,10.303565,20.607128,-2.811680
2,13.234580,80.277950,6.0,10.303566,30.910694,-2.811680
3,13.234550,80.277860,6.0,11.040572,41.951266,-2.554944
4,13.234495,80.277775,6.0,11.040573,52.991839,-2.554944
...,...,...,...,...,...,...
122,13.234601,80.278549,6.0,10.613892,1422.656061,3.013881
123,13.234614,80.278451,6.0,10.613891,1433.269952,3.013881
124,13.234626,80.278354,6.0,10.613891,1443.883843,3.013881
125,13.234638,80.278257,6.0,10.613890,1454.497733,3.013881


In [18]:
filtered_df = df.copy()
filtered_df['elevation'] = filtered_df['Elevation (m)']
filtered_df['step'] = filtered_df['step_distance']
filtered_df['direction'] = filtered_df['direction']
filtered_df['cum_distance'] = filtered_df['cum_distance']
filtered_df = filtered_df[['step', 'direction', 'elevation']]

In [29]:
LAPS = 220
lapped_df = pd.concat([filtered_df for _ in range(LAPS)], ignore_index=True)

lapped_df

,step,direction,elevation
0,10.303564,-2.811680,6.0
1,10.303565,-2.811680,6.0
2,10.303566,-2.811680,6.0
3,11.040572,-2.554944,6.0
4,11.040573,-2.554944,6.0
...,...,...,...
27935,10.613892,3.013881,6.0
27936,10.613891,3.013881,6.0
27937,10.613891,3.013881,6.0
27938,10.613890,3.013881,6.0


In [30]:
total_length = lapped_df['step'].sum()

# Desired number of rows (say 150)
num_chunks = 150
target_chunk_length = total_length / num_chunks

print(f'Total length: {total_length:.2f} m')
print(f'Target chunk length: {target_chunk_length:.2f} m')

# Container for downsampled rows
chunks = []

# Accumulators
current_step_sum = 0.0
current_direction_sum = 0.0
current_count = 0
last_elevation = 0.0

# For slope calc: previous chunk's end elevation
prev_elev = None

for _, row in lapped_df.iterrows():
    current_step_sum += row['step']
    current_direction_sum += row['direction']
    current_count += 1
    last_elevation = row['elevation']
    
    if current_step_sum >= target_chunk_length:
        avg_direction = current_direction_sum / current_count
        # If no previous elevation yet, slope = 0
        if prev_elev is None:
            slope = 0.0
        else:
            slope = (last_elevation - prev_elev) / current_step_sum
        chunks.append({
            'step': current_step_sum,
            'direction': avg_direction,
            'elevation': last_elevation,
            'slope': slope
        })
        # Update prev_elev for next chunk
        prev_elev = last_elevation
        # Reset accumulators for next chunk
        current_step_sum = 0.0
        current_direction_sum = 0.0
        current_count = 0

# Handle leftover
if current_count > 0:
    avg_direction = current_direction_sum / current_count
    if prev_elev is None:
        slope = 0.0
    else:
        slope = (last_elevation - prev_elev) / current_step_sum
    chunks.append({
        'step': current_step_sum,
        'direction': avg_direction,
        'elevation': last_elevation,
        'slope': slope
    })

# Convert to DataFrame
downsampled_df = pd.DataFrame(chunks)

Total length: 320745.10 m
Target chunk length: 2138.30 m


In [31]:
print(downsampled_df.head())
print(f'Downsampled rows: {len(downsampled_df)}')

# Save it
downsampled_df.to_csv('./data/lappedroute.csv', index=False)

          step  direction  elevation     slope
0  2148.737375   0.309732        7.0  0.000000
1  2147.327624   1.277951        6.0 -0.000466
2  2138.375586   0.412893        5.0 -0.000468
3  2140.930463   1.131366        4.0 -0.000467
4  2139.712724   0.562183        5.0  0.000467
Downsampled rows: 150
